In [105]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split
import time
from math import log10, floor
import os

from relational_rnn_models import RelationalMemory

In [96]:
# Define variables
n_vars = 3
var_ids = list(range(n_vars))
var_names = ['var' + str(i) for i in var_ids]
var_weights = [0.1, 0.6, 0.3] # variable distribution of mock data
n_time_steps = 6
n_individuals = 1000

noise_length = 2

In [3]:
# Helper function(s)

# round a number to n significant digits
def round_to_n(x, n = 2):
    return round(x, -int(floor(log10(abs(x)))) + (n - 1)) if x != 0 else 0

# visualize the output of the generator
def visualize_output(generator, z, n = 2):
    p = generator(z).view(n_time_steps, n_vars)
    p.shape
    for t in range(p.shape[0]):
        tmp = []
        for f in range(p.shape[1]):
            tmp.append(round_to_n(p[t,f], n))
        print(tmp)

In [14]:
# Generate mock data

events = []

start_time = time.time()

for indv in range(n_individuals):
    tmp = []
    for t in range(n_time_steps):
        if t > 0 and tmp[t - 1] == 'var2':
            weights = [0.7, 0.2, 0.1]
            var = np.random.choice(var_names, p=weights)
        else:
            var = np.random.choice(var_names, p=var_weights)
        tmp.append(var)
    events.append(tmp)
        
print('time taken:', round_to_n(time.time() - start_time), 'seconds')

for i in range(10):
    print(events[i])

In [44]:
vars_to_indices = dict([(v, i) for i, v in enumerate(var_names)])
print(vars_to_indices)
data = torch.tensor([[vars_to_indices[e] for e in event] for event in events])
data[:10]

{'var0': 0, 'var1': 1, 'var2': 2}


tensor([[0, 1, 1, 1, 2, 0],
        [0, 1, 2, 0, 1, 1],
        [1, 1, 2, 0, 2, 0],
        [1, 1, 1, 1, 1, 2],
        [2, 0, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1],
        [2, 1, 2, 0, 2, 0],
        [2, 0, 2, 0, 0, 0],
        [1, 1, 1, 2, 0, 1],
        [1, 1, 1, 2, 0, 1]])

In [100]:
# Define the generator

'''
class Generator(nn.Module):
    def __init__(self, mem_slots, head_size, num_heads, temperature, vocab_size, embed_vector_size, sequence_length):
        super(Generator, self).__init__()
        self.sequence_length = sequence_length
        
        self.embed = nn.Embedding(vocab_size, embed_vector_size)
        self.relational_memory = RelationalMemory(mem_slots, head_size, embed_vector_size, num_heads)
        
        self.temperature = temperature
        
        self.output_layer = nn.Linear(self.relational_memory.mem_size * self.relational_memory.mem_slots, vocab_size)
    
    # This function introduces the randomness into the generator; no need for additional random noise input?
    def gumbel_softmax(self, output, temperature, eps = 1e-10):
        u = torch.rand(output.shape)
        g = -torch.log(-torch.log(u + eps) + eps)
        result = torch.mul(output.add(g), temperature)
        result = F.softmax(result, dim=1)
        next_token = torch.argmax(result, dim=1)
        return result, next_token
    
    def forward_step(self):
        pass
    
    def forward(self, start_token):
        memory = self.relational_memory.initial_state(batch_size = 1)
        token = start_token
        for i in range(self.sequence_length):
            embed = self.embed(token)
            output, memory = self.relational_memory(embed, memory)
            output = self.output_layer(output)
            output, token = self.gumbel_softmax(output, self.temperature)
        return output, memory

'''


In [101]:
G = RelationalMemory(mem_slots = 4, head_size = 2, num_heads = 3, num_tokens = n_vars, temperature = 1, embed_vector_size = 2, sequence_length = n_time_steps)
#print(G.state_dict())
inp = data[0, :].view(1, -1)
#inp = torch.randn(1, n_time_steps, noise_length)
print(inp)
logits, memory = G(inp)
print(logits)


tensor([[[ 0.7778,  0.8254],
         [-0.9909,  0.1857],
         [-1.0551, -1.6426],
         [ 0.1763,  1.5427],
         [-0.1829, -0.3588],
         [-0.7206,  0.4743]]])
tensor([[[1., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0.]]])
tensor([[0.5593, 0.0915, 0.3492]], grad_fn=<SoftmaxBackward>)
tensor([[[-0.4666, -1.3479, -0.5847,  2.1674,  0.8708,  0.4461],
         [-0.5139,  2.5175, -0.0375,  1.5114,  0.0572,  0.2540],
         [-0.7707, -1.4941,  0.3154,  2.0375,  0.1664,  0.2389],
         [-0.8530, -1.9771, -0.8000,  2.7497,  0.9198,  0.4457]]],
       grad_fn=<AddBackward0>)


In [104]:
test = torch.tensor([[2]])
print(test.shape)
embed = nn.Embedding(10, 4)
test_embed = embed(test)
print(test_embed)
print(test_embed.shape)
test_inp = test_embed[:, 0]
print(test_inp.shape)
test_inp = test_inp.view(test_inp.shape[0], -1)
print(test_inp.shape)
test_inp.unsqueeze(dim=1).shape



torch.Size([1, 1])
tensor([[[-1.4959,  1.1291, -0.6207, -0.3380]]], grad_fn=<EmbeddingBackward>)
torch.Size([1, 1, 4])
torch.Size([1, 4])
torch.Size([1, 4])


torch.Size([1, 1, 4])

In [ ]:
# Define the Discriminator

class Discriminator(nn.Module):
    def __init__(self):
        pass